In [1]:
import os
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset

class InpaintingDataset(Dataset):
    def __init__(self, original_image_dir, mask_dir, transform_img=None, transform_mask=None):
        self.original_image_dir = original_image_dir
        self.mask_dir = mask_dir
        self.transform_img = transform_img
        self.transform_mask = transform_mask

        # 获取原始图像和掩码图像的路径
        self.image_paths = []
        for subdir in os.listdir(original_image_dir):
            image_subdir = os.path.join(original_image_dir, subdir)
            mask_subdir = os.path.join(mask_dir, subdir)          
            self.image_paths.append((image_subdir, mask_subdir))
        print(f"Loaded {len(self.image_paths)} image-mask pairs")


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path, mask_path = self.image_paths[idx]

        # 读取原始图像和掩码图像
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("1")  # mask是单通道图像

        # 应用数据增强和预处理
        if self.transform_img:
            img = self.transform_img(img)
        if self.transform_mask:
            mask = self.transform_mask(mask)
        
        return img, mask

transform_img = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),
])

# 为掩码图像定义数据增强（不进行归一化）
transform_mask = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [2]:
from diffusers import StableDiffusionInpaintPipeline
import torch

# 加载预训练模型
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "./stable-diffusion-2-inpainting",
    torch_dtype=torch.float16
)
pipe.to("cuda")


2024-12-31 23:56:40.658879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735660600.670221  174003 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735660600.673740  174003 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 23:56:40.687003: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

StableDiffusionInpaintPipeline {
  "_class_name": "StableDiffusionInpaintPipeline",
  "_diffusers_version": "0.31.0",
  "_name_or_path": "./stable-diffusion-2-inpainting",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
from torch.optim import AdamW

# 定义优化器
optimizer = AdamW(pipe.unet.parameters(), lr=1e-5, weight_decay=0.01)


In [4]:
import torch.nn as nn

# 定义损失函数
loss_fn = nn.MSELoss()


In [5]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Generator

generator = Generator()
generator.manual_seed(42)

def train_model(image_type, num_epochs=10, batch_size=16, learning_rate=1e-5):
    """
    训练针对特定图像类型的模型
    :param image_type: 图像类型 (例如 'face', 'scenario', 'street_scene_pairs', 'texture')
    :param num_epochs: 训练轮次
    :param batch_size: 批大小
    :param learning_rate: 学习率
    """
    # 构建当前类型的路径
    original_image_dir = f"./dataset_train/original_image/{image_type}"
    mask_dir = f"./dataset_train/mask/{image_type}"

    # 加载数据集
    train_dataset = InpaintingDataset(
        original_image_dir=original_image_dir,
        mask_dir=mask_dir,
        transform_img=transform_img,
        transform_mask=transform_mask
    )
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # 确保模型处于训练模式
    pipe.unet.train()

    # 创建优化器
    optimizer = optim.Adam(pipe.unet.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()  # 损失函数

    to_tensor = transforms.ToTensor()

    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} for {image_type}"):
            images, masks = batch
            images, masks = images.cuda(), masks.cuda()

            # 设置固定文本提示
            prompt = [" "] * images.size(0)

            # 使用 tokenizer 将文本提示转换为输入特征
            inputs = pipe.tokenizer(
                prompt,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
                max_length=pipe.tokenizer.model_max_length
            )

            # 将 tokenizer 的输出转到 GPU
            for k, v in inputs.items():
                inputs[k] = v.cuda()

            # 获取文本嵌入
            with torch.no_grad():
                prompt_embeds = pipe.text_encoder(
                    inputs.input_ids, attention_mask=inputs.attention_mask
                ).last_hidden_state

            # 前向传播
            outputs = pipe(
                image=images,
                mask_image=masks,
                prompt_embeds=prompt_embeds,
                generator=generator,
                num_inference_steps=50,
                guidance_scale=7.5
            ).images

            # 转换输出
            if isinstance(outputs, (list, tuple)):
                outputs = torch.stack([to_tensor(img) for img in outputs]).cuda()
            if not outputs.requires_grad:
                outputs = outputs.clone().requires_grad_(True)
            outputs = F.interpolate(outputs, size=(224, 224), mode='bilinear', align_corners=False)

            # 检查形状是否匹配
            if outputs.shape != images.shape:
                raise ValueError(f"Output shape {outputs.shape} does not match input shape {images.shape}")

            # 计算损失
            loss = loss_fn(outputs, images)
            print("!!!!!!!!!!!!", loss.item())

            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # 输出当前 epoch 的平均损失
        epoch_loss = running_loss / len(train_dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}] for {image_type}, Loss: {epoch_loss:.4f}")

    
    # 保存模型    
    model_path = f'unet_model_{image_type}.pth'
    torch.save(pipe.unet.state_dict(), model_path)
    print(f"Model for {image_type} saved to {model_path}")

In [6]:
###### 依次训练每种类型的模型
image_types = ["face", "scenario", "street_scene_pairs", "texture"]
for image_type in image_types:
    train_model(image_type=image_type, num_epochs=10, batch_size=16, learning_rate=1e-5)

Loaded 800 image-mask pairs


Epoch 1/10 for texture:   0%|                            | 0/50 [00:00<?, ?it/s]/home/neonsun/miniconda3/envs/sd/lib/python3.10/site-packages/diffusers/image_processor.py:704: FutureWarning: Passing `image` as torch tensor with value range in [-1,1] is deprecated. The expected value range for image tensor is [0,1] when passing as pytorch tensor or numpy Array. You passed `image` with value range [-1.7922625541687012,2.1458969116210938]
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/10 for texture:   2%|▎                 | 1/50 [01:13<1:00:06, 73.61s/it]

!!!!!!!!!!!! 0.6305872797966003


...

!!!!!!!!!!!! 0.6802368760108948


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 10/10 for texture: 100%|████████████████| 50/50 [1:01:36<00:00, 73.92s/it]

!!!!!!!!!!!! 0.824164867401123
Epoch [10/10] for texture, Loss: 0.8375


Model for texture saved to unet_model_texture.pth
